In [ ]:
# Mount to Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Define Project Folder
FOLDERNAME = 'Colab\ Notebooks/SC201L16'

%cd drive/MyDrive/$FOLDERNAME

In [2]:
# Define device
import torch
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print('Device:', device)

Device: cuda


In [3]:
# Load Existing Dataset
import torchvision.datasets as dset
import torchvision.transforms as T
transform = T.Compose([T.ToTensor(),T.RandomAutocontrast()])
train_data = dset.MNIST('./train',train=True,download=True,transform=transform)
val_data = dset.MNIST('./val',train=False,download=True,transform=transform)

In [4]:
# Check Data Dimension
print(train_data)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./train
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               RandomAutocontrast(p=0.5)
           )


In [5]:

train_data[0][0].shape ## print data

torch.Size([1, 28, 28])

In [6]:
num_train = len(train_data)
num_val = len(val_data)
print('Number of training:', num_train)
print('Number of validation:', num_val)

Number of training: 60000
Number of validation: 10000


In [7]:
# Build Model
import torch.nn as nn
model = None
model = nn.Sequential(
    # N x 1 x 28 x 28

    nn.Conv2d(in_channels=1,out_channels=64,kernel_size=3,padding=1,stride =1),
    nn.BatchNorm2d(num_features=64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2,stride=2),
    # 16 x 64 x 14 x 14

    nn.Flatten(),
    # 10 neurons(0-9)
    nn.Linear(64*14*14,10)
)

In [8]:
# Move model to GPU
# -> GPU
model = model.cuda()

In [9]:
# Create Mini-batches
from torch.utils.data import DataLoader
mini_trains = DataLoader(train_data, batch_size=128, shuffle=True)
mini_vals = DataLoader(val_data, batch_size=128, shuffle=True)

In [10]:
# Training Procedure
def train(num_epoch, model, mini_trains, mini_vals, device, loss_function, optimizer):
  for epoch in range(num_epoch):
    num_iters = 0
    for x, y in mini_trains:
      model.train()
      x = x.to(device)
      y = y.to(device)
      scores = model(x)
      loss = loss_function(scores, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      if num_iters % 10 == 0:
        evaluate_predictor(model, epoch, mini_vals, device)
      num_iters += 1

In [11]:
# Validating Procedure
def evaluate_predictor(model, epoch, mini_vals, device):
  model.eval()
  with torch.no_grad():
    acc_count = 0
    for x, y in mini_vals:
      x = x.to(device)
      y = y.to(device)
      scores=model(x)
      predictions=scores.max(1)[1]
      acc = predictions.eq(y).sum().item()
      acc_count += acc
    print(f'Epoch[{epoch+1}] Acc: {acc_count/num_val}')

In [12]:
# Define loss function & optimizer
import torch.optim as optim
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [13]:
# Start training
train(1, model, mini_trains, mini_vals, device, loss_function, optimizer)

Epoch[1] Acc: 0.2338
Epoch[1] Acc: 0.7182
Epoch[1] Acc: 0.8336
Epoch[1] Acc: 0.8902
Epoch[1] Acc: 0.8941
Epoch[1] Acc: 0.9185
Epoch[1] Acc: 0.9286
Epoch[1] Acc: 0.9247
Epoch[1] Acc: 0.9323
Epoch[1] Acc: 0.9366
Epoch[1] Acc: 0.9416
Epoch[1] Acc: 0.9466
Epoch[1] Acc: 0.9418
Epoch[1] Acc: 0.9459
Epoch[1] Acc: 0.9502
Epoch[1] Acc: 0.958
Epoch[1] Acc: 0.9582
Epoch[1] Acc: 0.9494
Epoch[1] Acc: 0.9608
Epoch[1] Acc: 0.9605
Epoch[1] Acc: 0.9549
Epoch[1] Acc: 0.9622
Epoch[1] Acc: 0.9646
Epoch[1] Acc: 0.9642
Epoch[1] Acc: 0.9632
Epoch[1] Acc: 0.9597
Epoch[1] Acc: 0.9675
Epoch[1] Acc: 0.9632
Epoch[1] Acc: 0.9573
Epoch[1] Acc: 0.9661
Epoch[1] Acc: 0.9668
Epoch[1] Acc: 0.9681
Epoch[1] Acc: 0.9676
Epoch[1] Acc: 0.9651
Epoch[1] Acc: 0.9694
Epoch[1] Acc: 0.9675
Epoch[1] Acc: 0.9663
Epoch[1] Acc: 0.9712
Epoch[1] Acc: 0.9702
Epoch[1] Acc: 0.9686
Epoch[1] Acc: 0.9573
Epoch[1] Acc: 0.9701
Epoch[1] Acc: 0.9721
Epoch[1] Acc: 0.9695
Epoch[1] Acc: 0.9751
Epoch[1] Acc: 0.9697
Epoch[1] Acc: 0.9716
